# About
In recent years, Cloud Model 1 (CM1; http://www2.mmm.ucar.edu/people/bryan/cm1/) has become a very popular tool for performing idealized studies of atmospheric phenomena. There exists very little support for computing trajectories using CM1 output, which are usually necessary to understand the processes of the atmospheric phenomena of interest. Natively, CM1 only supports 'online' forward trajectories in 2D simulations and in 3D simulation without terrain. I wrote this script because there are no adequate tools available to compute highly customizable 'offline' trajectories in simulations with terrain. This script is intended to be easily customizable.

Notes:

* Can compute backward or forward trajectories (Default is backward, but can be forward with simple changes to "Calculate Trajectories" block)
* Written to work with 3D model output (can be modified to work with 2D output)
* Will work with or without terrain
* Initial location, number, and density of parcels can be easily specified in "Initialize Parcels" block
* Uses xarray and Dask to distribute memory and calculation across multiple processors
* With modifications, can be used with WRF output (several others have already done so)
* Comments that say "set by user" are specific to model output and desired trajectories

# Load Modules

In [56]:
import os, sys
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate
import time
import scipy.ndimage

%config IPCompleter.greedy=True

# Read in CM1 Output

* User must insert path to data
    * If model output is one file use ***xr.open_dataset***
    * If model output is in multiple files use ***xr.openmfdataset***


In [57]:
#Use xarray to open model output and specify chunking if data set is large (set by user)
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group8/tom/cm1/output/12ms_2000m_japan.nc', chunks={'nk': 1})

#Get model output dimensions
num_x = ds.nx #Number of gridpoints in x
num_y = ds.ny #Number of gridpoints in y
num_z = ds.nz #Number of gridpoints in z

x = np.arange(0,num_x,1)
y = np.arange(0,num_y,1)
z = np.arange(0,num_z,1)

# Create Dask Cluster

In [58]:
#Option to use multiple processors and threads (set by user)
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers=10, threads_per_worker=2)
client = Client(c)
client

Client Scheduler: tcp://127.0.0.1:43510 Dashboard: http://127.0.0.1:38664/status,Cluster Workers: 10 Cores: 20 Memory: 48.24 GB


# Initialize Parcels

User must enter desired trajectory characteristics

In [59]:
#Number of parcels in vertical (can be more than number of vertical levels; set by user) 
num_seeds_z = 1

#Number of parcels in y (set by user) 
num_seeds_y = 30

#Number of time steps to run trajectories forward (set by user) 
time_steps = 50
incre = 5

#Time step to start forwards trajectories at (set by user) 
start_time_step = 300

#Variable to record at each parcel's location throughout trajectory (code can be easily modified to add more; set by user) 
var_name1 = 'th'

#Set as 'Y' or 'N' for 'yes' or 'no' if the u, v, and w model output is on the staggered grid 
#(unless you have interpolated u, v, and w to the scalar grid, they are most likely on the staggered grid (set by user)
staggered = 'N'

**Model output info**

In [60]:
#Horizontal resolution of model output (meters)
hor_resolution = (ds.xf[1].values-ds.xf[0].values)*1000

#Vertical resolution of model output (meters). Changes in x and y, if there is terrain, and z, if grid is stretched.
try:
    vert_resolution = ds.zh[0,1:,:,:].values-ds.zh[0,:-1,:,:].values
    print('Output has terrain')
except:
    vert_res1d = (ds.z[1:].values-ds.z[:-1].values)*1000
    vert_res2d = np.repeat(vert_res1d,ds.ny, axis = 0).reshape(ds.nz-1, ds.ny)
    vert_resolution = np.repeat(vert_res2d,ds.nx, axis = 0).reshape(ds.nz-1, ds.ny, ds.nx)
    print('Output does not have terrain')
                  
#Model output time step length (seconds)
time_step_length = (ds.time[1].values - ds.time[0].values)/np.timedelta64(1, 's')*incre

Output has terrain


**Create empty arrays to store x, y, and z positions of parcels**

In [61]:
xpos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #x-location (grid points on staggered grid)
ypos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #y-location (grid points on staggered grid)
zpos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #z-location (grid points on staggered grid)
zpos_heightASL = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #Height above sea level (meters)
zpos_vert_res = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #Vertical grid spacing at parcel location (meters)
variable1 = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #User specified variable to track

**Initial location of parcels in gridpoints, specifically on the scalar grid (set by user). Initializes an array of parcels in the the y-z domain (modification necessary for x-dimension or 3D array of parcels)**

In [62]:
#x-position
xpos[0,:,:] = 1600 #This example initializes all seeds at same x-position (1000th x-grpt, set by user)

#y-position   
for i in range(num_seeds_y):
    ypos[0,:,i] = 50 + (30*i) #This example initializes seeds evenly in y-dimension (0th, 4th, 8th, etc. y-grpt; set by user)

#z-position
for i in range(num_seeds_z):
    zpos[0,i,:] = 0 #This example initializes seeds evenly starting in z-dimension (0th, 1st, 2nd, etc., z-grpt; set by user)

## Determine Initial Height of Parcels Above Sea Level
Use the height of the models levels (meters above sea level) to convert from terrain following grid points to height above seal level.

In [63]:
#Get height of surface
try:
    zs = ds.zh[0,:,:].values
    print('Output has terrain')
except: 
    zs = np.zeros((ds.ny,ds,nx))
    print('Output does not have terrain')

#Get height of vertical coordinates (scalar grid)
try:
    zh = ds.zh[0,:,:,:].values
    print('Output has terrain')
except:
    zh1d = (ds.z[:].values)*1000
    zh2d = np.repeat(zh1d,ds.ny, axis = 0).reshape(ds.nz, ds.ny)
    zh = np.repeat(zh2d,ds.nx, axis = 0).reshape(ds.nz, ds.ny, ds.nx)
    print('Output does not have terrain')

#Create list of initial coordinates to get height
xloc = (xpos[0,:,:]).flatten()
yloc = (ypos[0,:,:]).flatten()
zloc = (zpos[0,:,:]).flatten()
coord_height = []
for i in range(len(xloc)):
    coord_height.append((zloc[i], yloc[i], xloc[i]))

#Get the actual inital height of the parcels in meters above sea level
zpos_heightASL[0,:,:] = np.reshape(interpolate.interpn((z,y,x), zh, coord_height, method='linear', bounds_error=False, fill_value= 0), (num_seeds_z, num_seeds_y))

Output has terrain


# Calculate Trajectories
The method used to calculate trajectories is a second-order semi-implicit discretization in space and time. The method is described in depth in section 2.1 pf Miltenberger et al. 2013 (https://www.geosci-model-dev.net/6/1989/2013/gmd-6-1989-2013.pdf)
Unless user is changing trajectories from backwards to forwards, nothing should be changed here.

In [64]:
#Loop over all time steps and compute trajectory
for t in range(time_steps-1):
    
    start = time.time() #Timer
    
    ##########################################################################################################
    ##########################################################################################################   
    ##################### Get data for 'first guess' step of integration scheme ##############################
    ##########################################################################################################
    ##########################################################################################################
    
    xmin = np.int(np.nanmin(xpos[t,:,:])-2)
    xmin = 0 if xmin < 0 else xmin
    
    xmax = np.int(np.nanmax(xpos[t,:,:])+2)
    xmax = ds.nx if xmax > ds.nx else xmax
    
    ymin = np.int(np.nanmin(ypos[t,:,:])-2)
    ymin = 0 if ymin < 0 else ymin
    
    ymax = np.int(np.nanmax(ypos[t,:,:])+2)
    ymax = ds.ny if ymax > ds.ny else ymax
    
    zmin = np.int(np.nanmin(zpos[t,:,:])-2)
    zmin = 0 if zmin < 0 else zmin
    
    zmax = np.int(np.nanmax(zpos[t,:,:])+2)
    zmax = ds.nz if zmax > ds.nz else zmax
    
    x_fast = np.arange(0,xmax-xmin)
    y_fast = np.arange(0,ymax-ymin)
    z_fast = np.arange(0,zmax-zmin)
    
    #Get model data
    t = np.int(t * incre)
    
    u = ds.uinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    v = ds.vinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    w = ds.winterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    var1 = getattr(ds,var_name1)[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    
    t = np.int(t/incre) 
    
    #Smooth data
    smooth = 20
    u = np.copy(scipy.ndimage.filters.uniform_filter(u, smooth))
    v = np.copy(scipy.ndimage.filters.uniform_filter(v, smooth))
    w = np.copy(scipy.ndimage.filters.uniform_filter(w, smooth))
    
    
    #Get surface height grid (set to zero if no terrain)
    try:
        zs = np.array(ds.zs[0,:,:])
    except:
        zs = np.zeros((ds.ny, ds.nx))  
        
        
    ############## Generate coordinates for interpolations ###############

    #x, y, and z on staggered and scalar grids
    xloc = np.copy(xpos[t,:,:]).flatten()-xmin
    xloc_stag = np.copy(xpos[t,:,:]+0.5).flatten()-xmin
    yloc = np.copy(ypos[t,:,:]).flatten()-ymin
    yloc_stag = np.copy(ypos[t,:,:]+0.5).flatten()-ymin
    zloc = np.copy(zpos[t,:,:]).flatten()-zmin
    zloc_stag = np.copy(zpos[t,:,:]+0.5).flatten()-zmin

    #If u, v, and w are staggered, generate three staggered sets of coordinates:
    #    1) u-grid (staggered in x)
    #    2) v-grid (staggered in y)
    #    3) w-grid (staggered in z)
    
    if staggered == 'Y':
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc_stag[i])) 
            coord_v.append((zloc[i], yloc_stag[i], xloc[i])) 
            coord_w.append((zloc_stag[i], yloc[i], xloc[i])) 
    
    #If not, generate scalar coordinates
    else: 
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc[i])) 
            coord_v.append((zloc[i], yloc[i], xloc[i])) 
            coord_w.append((zloc[i], yloc[i], xloc[i])) 
    
    #Scalar coordinates for all other variables
    coord = []
    coord_fast = []
    for i in range(len(xloc)):
        coord.append((zloc[i]+zmin, yloc[i]+ymin, xloc[i]+xmin)) 
        coord_fast.append((zloc[i], yloc[i], xloc[i])) 
    
    
    
    ##########################################################################################################   
    ########################## Integrate 'first guess' of parcel's new location ##############################
    ##########################################################################################################   

    
    #########################   Calc 'first guess' new xpos in grdpts   #######################################
    dx_0 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), u, coord_u, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))
    xpos_1 = xpos[t,:,:] + dx_0

    #########################   Calc 'first guess' new ypos in grdpts   #######################################
    dy_0 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), v, coord_v, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))
    ypos_1 = ypos[t,:,:] + dy_0

    #########################   Calc 'first guess' new zpos in meters above sea level ######################################
    dz_0 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), w, coord_w, method='linear', bounds_error=False, fill_value= 0)*time_step_length, (num_seeds_z, num_seeds_y))
    zpos_heightASL_1 = zpos_heightASL[t,:,:] + dz_0
    
    ############# Convert zpos from meters above sea level to gridpts abve surface for interpolation #########
    #Get vertical grid spacing at each parcel's location
    zpos_vert_res[t,:,:] = np.reshape(interpolate.interpn((z[:-1],y,x), vert_resolution, coord, method='linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))


    #Calculate change in surface height and change in parcel height
    xloc = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]).flatten()
    coord_zs1 = []
    for i in range(len(xloc)):
        coord_zs1.append((yloc[i], xloc[i]))
        
    xloc = np.copy(xpos_1).flatten()
    yloc = np.copy(ypos_1).flatten()
    coord_zs2 = []
    for i in range(len(xloc)):
        coord_zs2.append((yloc[i], xloc[i]))
    
    #Change in surface height over last timestep
    zs1 = interpolate.interpn((y,x), zs, coord_zs1, method='linear', bounds_error=False, fill_value= np.nan)
    zs2 = interpolate.interpn((y,x), zs, coord_zs2, method='linear', bounds_error=False, fill_value= np.nan)
    zs_change = zs2-zs1
    
    #Change in parcel height over last times step
    zpos_heightASL_change = zpos_heightASL_1.flatten()-zpos_heightASL[t,:,:].flatten()
    
    #Calculate zpos in grdpts above surface
    zpos_1 = zpos[t,:,:] + np.reshape((zpos_heightASL_change - zs_change)/zpos_vert_res[t,:,:].flatten(), (num_seeds_z, num_seeds_y))
    
    #Prevent parcels from going into the ground
    zpos_heightASL_1 = zpos_heightASL_1.clip(min=0)
    zpos_1 = zpos_1.clip(min=0)


    #Calculate value of variable at each parcel's location
    variable1[t,:,:] = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), var1, coord_fast, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y)) 

    ##########################################################################################################
    ##########################################################################################################   
    ##################### Get data for 'correction' step of integration scheme ###############################
    ##########################################################################################################
    ##########################################################################################################
    
    
    xmin = np.int(np.nanmin(xpos_1)-2)
    xmin = 0 if xmin < 0 else xmin
    
    xmax = np.int(np.nanmax(xpos_1)+2)
    xmax = ds.nx if xmax > ds.nx else xmax
    
    ymin = np.int(np.nanmin(ypos_1)-2)
    ymin = 0 if ymin < 0 else ymin
    
    ymax = np.int(np.nanmax(ypos_1)+2)
    ymax = ds.ny if ymax > ds.ny else ymax
    
    zmin = np.int(np.nanmin(zpos_1)-2)
    zmin = 0 if zmin < 0 else zmin
    
    zmax = np.int(np.nanmax(zpos_1)+2)
    zmax = ds.nz if zmax > ds.nz else zmax
    
    x_fast = np.arange(0,xmax-xmin)
    y_fast = np.arange(0,ymax-ymin)
    z_fast = np.arange(0,zmax-zmin)
    
    #Get model data for next time step
    t = t + 1
    t = np.int(t * incre)
    
    u = ds.uinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    v = ds.vinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    w = ds.winterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    
    t = np.int(t/incre)
    t = t - 1
    
    #Smooth data
    smooth = 20
    u = np.copy(scipy.ndimage.filters.uniform_filter(u, smooth))
    v = np.copy(scipy.ndimage.filters.uniform_filter(v, smooth))
    w = np.copy(scipy.ndimage.filters.uniform_filter(w, smooth))
    var1 = np.copy(scipy.ndimage.filters.uniform_filter(var1, smooth))
    
    
    #Get surface height grid (set to zero if no terrain)
    try:
        zs = np.array(ds.zs[0,:,:])
    except:
        zs = np.zeros((ds.ny, ds.nx))  
        
        
    ############## Generate coordinates for interpolations ###############

    #x, y, and z on staggered and scalar grids
    xloc = np.copy(xpos_1).flatten()-xmin
    xloc_stag = np.copy(xpos_1+0.5).flatten()-xmin
    yloc = np.copy(ypos_1).flatten()-ymin
    yloc_stag = np.copy(ypos_1+0.5).flatten()-ymin
    zloc = np.copy(zpos_1).flatten()-zmin
    zloc_stag = np.copy(zpos_1+0.5).flatten()-zmin

    #If u, v, and w are staggered, generate three staggered sets of coordinates:
    #    1) u-grid (staggered in x)
    #    2) v-grid (staggered in y)
    #    3) w-grid (staggered in z)
    
    if staggered == 'Y':
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc_stag[i])) 
            coord_v.append((zloc[i], yloc_stag[i], xloc[i])) 
            coord_w.append((zloc_stag[i], yloc[i], xloc[i])) 
    
    #If not, generate scalar coordinates
    else: 
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc[i])) 
            coord_v.append((zloc[i], yloc[i], xloc[i])) 
            coord_w.append((zloc[i], yloc[i], xloc[i])) 
    
    #Scalar coordinates for all other variables
    coord = []
    coord_fast = []
    for i in range(len(xloc)):
        coord.append((zloc[i]+zmin, yloc[i]+ymin, xloc[i]+xmin)) 
        coord_fast.append((zloc[i], yloc[i], xloc[i])) 
    
    
    
    
    ##########################################################################################################   
    ########################## Integrate 'correction' of parcel's new location ###############################
    ##########################################################################################################   

    
    #########################   Calc 'correction' new xpos in grdpts   #######################################
    dx_1 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), u, coord_u, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))
    xpos[t+1,:,:] = xpos[t,:,:] + (dx_0 + dx_1)/2

    #########################   Calc 'correction' new ypos in grdpts   #######################################
    dy_1 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), v, coord_v, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))
    ypos[t+1,:,:] = ypos[t,:,:] + (dy_0 + dy_1)/2

    #########################   Calc 'correction' new zpos in meters above sea level ######################################
    dz_1 = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), w, coord_w, method='linear', bounds_error=False, fill_value= 0)*time_step_length, (num_seeds_z, num_seeds_y))
    zpos_heightASL[t+1,:,:] = zpos_heightASL[t,:,:] + (dz_0 + dz_1)/2
    
    
    
    ############# Convert zpos from meters above sea level to gridpts abve surface for interpolation #########
    #Get vertical grid spacing at each parcel's location
    zpos_vert_res[t,:,:] = np.reshape(interpolate.interpn((z[:-1],y,x), vert_resolution, coord, method='linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))

 
    #Calculate change in surface height and change in parcel height
    xloc = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]).flatten()
    coord_zs1 = []
    for i in range(len(xloc)):
        coord_zs1.append((yloc[i], xloc[i]))
        
    xloc = np.copy(xpos[t+1,:,:]).flatten()
    yloc = np.copy(ypos[t+1,:,:]).flatten()
    coord_zs2 = []
    for i in range(len(xloc)):
        coord_zs2.append((yloc[i], xloc[i]))
    
    #Change in surface height over last timestep
    zs1 = interpolate.interpn((y,x), zs, coord_zs1, method='linear', bounds_error=False, fill_value= np.nan)
    zs2 = interpolate.interpn((y,x), zs, coord_zs2, method='linear', bounds_error=False, fill_value= np.nan)
    zs_change = zs2-zs1
    
    #Change in parcel height over last times step
    zpos_heightASL_change = zpos_heightASL[t+1,:,:].flatten()-zpos_heightASL[t,:,:].flatten()
    
    #Calculate zpos in grdpts above surface
    zpos[t+1,:,:] = zpos[t,:,:] + np.reshape((zpos_heightASL_change - zs_change)/zpos_vert_res[t,:,:].flatten(), (num_seeds_z, num_seeds_y))
    ##########################################################################################################
    
    
    #Prevent parcels from going into the ground
    zpos = zpos.clip(min=0)
    zpos_heightASL = zpos_heightASL.clip(min=0)
    
 
    
    #Timer
    stop = time.time()
    print("Integration {:01d} took {:.2f} seconds".format(t, stop-start))
    
    

Integration 0 took 5.60 seconds
Integration 1 took 4.02 seconds
Integration 2 took 3.60 seconds
Integration 3 took 3.93 seconds
Integration 4 took 4.41 seconds
Integration 5 took 4.73 seconds
Integration 6 took 5.82 seconds
Integration 7 took 6.14 seconds
Integration 8 took 6.89 seconds
Integration 9 took 8.13 seconds
Integration 10 took 7.86 seconds
Integration 11 took 7.87 seconds
Integration 12 took 15.92 seconds
Integration 13 took 7.79 seconds
Integration 14 took 8.79 seconds
Integration 15 took 8.58 seconds
Integration 16 took 8.72 seconds
Integration 17 took 9.23 seconds
Integration 18 took 8.42 seconds
Integration 19 took 9.20 seconds
Integration 20 took 10.07 seconds
Integration 21 took 9.93 seconds
Integration 22 took 10.08 seconds
Integration 23 took 11.05 seconds
Integration 24 took 10.69 seconds
Integration 25 took 9.25 seconds
Integration 26 took 9.84 seconds
Integration 27 took 8.71 seconds
Integration 28 took 10.04 seconds
Integration 29 took 9.67 seconds
Integration 30

IndexError: index -2 is out of bounds for axis 1 with size 0

Get variable data for final time step

In [65]:
t = time_steps-1
var1 = getattr(ds,var_name1)[start_time_step-t,:,:,:].values

#Get get x, y, and z positions from scalar grid
xloc = np.copy(xpos[t,:,:]-0.5).flatten()
yloc = np.copy(ypos[t,:,:]-0.5).flatten()
zloc = np.copy(zpos[t,:,:]-0.5).flatten()
coord = []
for i in range(len(xloc)):
    coord.append((zloc[i], yloc[i], xloc[i])) 

#Variables
variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coord, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))

# Save Trajectory Data
The x, y, and z positions and user-specified variable values are saved in 3D numpy arrays. The first dimension is time and the other two are the positions and values of variables of all the parcels at that specifc time.

In [66]:
np.save('trajectory_arrays/xpos_2000m_soj', xpos)
np.save('trajectory_arrays/ypos_2000m_soj', ypos)
np.save('trajectory_arrays/zpos_2000m_soj', zpos_heightASL)
np.save('trajectory_arrays/%s_2000m_soj' %var_name1, variable1)